In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
dataset_url = "https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification"
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alikaggaventador31
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification


100%|██████████| 82.0M/82.0M [00:00<00:00, 1.37GB/s]

In [4]:
from torchvision import datasets, transforms
df = "/content/garbage-classification/Garbage classification/Garbage classification"
train_transformer = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = datasets.ImageFolder(df, transform=train_transformer)
print(len(dataset))

2537


In [5]:
import torch

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import os
import matplotlib.pyplot as plt

In [9]:
train_set, test_set = torch.utils.data.random_split(dataset, [2028, 509])
test_set.dataset.transform = test_transforms
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

In [10]:
model = models.resnet50(weights="DEFAULT")
for param in model.parameters():
  param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 6)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 145MB/s]


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [12]:
!pip install torchinfo

In [13]:
from torchinfo import summary
summary(model, input_size=(32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 6]                   --
├─Conv2d: 1-1                            [32, 64, 112, 112]        (9,408)
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        (128)
├─ReLU: 1-3                              [32, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [32, 64, 56, 56]          --
├─Sequential: 1-5                        [32, 256, 56, 56]         --
│    └─Bottleneck: 2-1                   [32, 256, 56, 56]         --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          (4,096)
│    │    └─BatchNorm2d: 3-2             [32, 64, 56, 56]          (128)
│    │    └─ReLU: 3-3                    [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                  [32, 64, 56, 56]          (36,864)
│    │    └─BatchNorm2d: 3-5             [32, 64, 56, 56]          (128)
│    │    └─ReLU: 3-6                    [32, 64, 56, 56]   

In [14]:
for epoch in range(5):
  model.train()
  running_loss = 0
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f"Epoch {epoch+1} loss: {running_loss/len(train_loader)}")

Epoch 1 loss: 1.1527161924168468
Epoch 2 loss: 0.6864866046234965
Epoch 3 loss: 0.5382186435163021
Epoch 4 loss: 0.4525669780559838
Epoch 5 loss: 0.3993573575280607


In [15]:
for param in model.parameters():
  param.requires_grad = True
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [16]:
for epoch in range(15):
  model.train()
  running_loss = 0
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f"Epoch {epoch+1} loss: {running_loss/len(train_loader)}")

Epoch 1 loss: 0.31034075608476996
Epoch 2 loss: 0.20764028921257704
Epoch 3 loss: 0.14647123671602458
Epoch 4 loss: 0.10656312329228967
Epoch 5 loss: 0.0787513300892897
Epoch 6 loss: 0.05696166108828038
Epoch 7 loss: 0.04243719970691018
Epoch 8 loss: 0.034045730004436336
Epoch 9 loss: 0.030994846791145392
Epoch 10 loss: 0.0254764988203533
Epoch 11 loss: 0.02236729320429731
Epoch 12 loss: 0.01748510503966827
Epoch 13 loss: 0.016821239121782128
Epoch 14 loss: 0.015509181521338178
Epoch 15 loss: 0.014021847993717529


In [17]:
model.eval()
correction = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correction += (predicted == labels).sum().item()
print(f"Accuracy: {100*correction/total}")

Accuracy: 90.96267190569745


In [18]:
classes = train_set.dataset.classes
print(classes)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [19]:
model_name = "garbage_classifier.pth"
checkpoint = {
    'model_state_dict': model.state_dict(),
    'classes': classes,
    'architecture': 'resnet18'
}
torch.save(checkpoint, model_name)
print(f"Modèle sauvegardé sous : {model_name}")
print(f"Classes sauvegardées : {classes}")

Modèle sauvegardé sous : garbage_classifier.pth
Classes sauvegardées : ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [20]:
from google.colab import files
files.download(model_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>